In [1]:
SELECT top 10 *
FROM Cyclistic_202109

(10 rows affected)

Total execution time: 00:00:00.008

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,F16,number_members,number_casual,mean_ride_length,max_ride_length,mode_day
B89639535028F8A8,classic_bike,2021-09-12 17:49:57.000,2021-09-12 19:41:25.000,Pine Grove Ave & Irving Park Rd,TA1308000022,Pine Grove Ave & Irving Park Rd,TA1308000022,41.954383,-87.648043,41.954383,-87.648043,casual,1899-12-30 01:51:28.000,1,NULL,NULL,NULL,NULL,NULL,NULL
869F9B901520E6F4,electric_bike,2021-09-06 17:27:31.000,2021-09-06 18:28:03.000,Troy St & North Ave,15653,Troy St & North Ave,15653,41.9098351666666,-87.7053775,41.9098543333333,-87.7053835,casual,1899-12-30 01:00:32.000,2,NULL,NULL,NULL,NULL,NULL,NULL
414BED1B55D60675,classic_bike,2021-09-27 13:41:40.000,2021-09-27 14:18:22.000,Marine Dr & Ainslie St,KA1504000171,Marine Dr & Ainslie St,KA1504000171,41.9716,-87.650154,41.9716,-87.650154,casual,1899-12-30 00:36:42.000,2,NULL,NULL,NULL,NULL,NULL,NULL
90EDB547332E2070,classic_bike,2021-09-11 15:38:56.000,2021-09-11 15:51:21.000,Pine Grove Ave & Irving Park Rd,TA1308000022,Marine Dr & Ainslie St,KA1504000171,41.954383,-87.648043,41.9716,-87.650154,casual,1899-12-30 00:12:25.000,7,NULL,NULL,NULL,NULL,NULL,NULL
57D7B7C12747009D,classic_bike,2021-09-19 19:55:16.000,2021-09-19 19:58:47.000,Sheffield Ave & Webster Ave,TA1309000033,Bissell St & Armitage Ave,13059,41.92154,-87.653818,41.91844,-87.65222,casual,1899-12-30 00:03:31.000,1,NULL,NULL,NULL,NULL,NULL,NULL
F9B959923F25703B,classic_bike,2021-09-20 17:46:40.000,2021-09-20 18:13:55.000,Troy St & North Ave,15653,Troy St & North Ave,15653,41.90976930169,-87.7052804871,41.90976930169,-87.7052804871,casual,1899-12-30 00:27:15.000,2,NULL,NULL,NULL,NULL,NULL,NULL
E3543080D97338EE,classic_bike,2021-09-19 09:51:32.000,2021-09-19 10:02:07.000,Marine Dr & Ainslie St,KA1504000171,Marine Dr & Ainslie St,KA1504000171,41.9716,-87.650154,41.9716,-87.650154,casual,1899-12-30 00:10:35.000,1,NULL,NULL,NULL,NULL,NULL,NULL
31572DF77EA77868,classic_bike,2021-09-26 13:06:39.000,2021-09-26 13:18:04.000,Marine Dr & Ainslie St,KA1504000171,Marine Dr & Ainslie St,KA1504000171,41.9716,-87.650154,41.9716,-87.650154,casual,1899-12-30 00:11:25.000,1,NULL,NULL,NULL,NULL,NULL,NULL
A529E2EDA96B4CA2,classic_bike,2021-09-06 12:14:05.000,2021-09-06 12:48:40.000,Troy St & North Ave,15653,Troy St & North Ave,15653,41.90976930169,-87.7052804871,41.90976930169,-87.7052804871,casual,1899-12-30 00:34:35.000,2,NULL,NULL,NULL,NULL,NULL,NULL
7147006750C9E020,classic_bike,2021-09-05 19:02:01.000,2021-09-05 19:26:40.000,Marine Dr & Ainslie St,KA1504000171,Marine Dr & Ainslie St,KA1504000171,41.9716,-87.650154,41.9716,-87.650154,casual,1899-12-30 00:24:39.000,1,NULL,NULL,NULL,NULL,NULL,NULL


In [3]:
-------------------------------------- Create a CTE of yearly data by combining all data for all months --------------------------------------

WITH yearly_data as 
(
SELECT *
FROM Cyclistic_202109
UNION all
SELECT *
FROM Cyclistic_202110
UNION all
SELECT *
FROM Cyclistic_202111
UNION all
SELECT *
FROM Cyclistic_202112
UNION all
SELECT *
FROM Cyclistic_202201
UNION all
SELECT *
FROM Cyclistic_202202
UNION all
SELECT *
FROM Cyclistic_202203
UNION all
SELECT *
FROM Cyclistic_202204
UNION all
SELECT *
FROM Cyclistic_202205
UNION all
SELECT *
FROM Cyclistic_202206
UNION all
SELECT *
FROM Cyclistic_202207
UNION all
SELECT *
FROM Cyclistic_202208
),


-------------------------------------- Remove all data points with NULL values --------------------------------------

NULL_cleaned AS
(
	SELECT *
	FROM yearly_data WHERE start_station_name   NOT LIKE '%NULL%'
						AND end_station_name  NOT LIKE '%NULL%'
							AND start_lat  NOT LIKE '%NULL%'
								AND start_lng  NOT LIKE '%NULL%'
									AND end_lat  NOT LIKE '%NULL%'
										AND end_lng NOT LIKE'%NULL%'
											AND start_station_id NOT LIKE'%NULL%'
),


-------------------------------------- Calculate the total ride length in minutes and assign day of the weel names --------------------------------------

aggre_data AS (
	SELECT *,
    DATEDIFF(MINUTE,started_at, ended_at) as TotalMinutes,
    CASE
      WHEN day_of_week = 2 THEN 'Monday'
      WHEN day_of_week = 3 THEN 'Tuesday'
      WHEN day_of_week = 4 THEN 'Wednesday'
      WHEN day_of_week = 5 THEN 'Thursday'
      WHEN day_of_week = 6 THEN 'Friday'
      WHEN day_of_week = 7 THEN 'Saturday'
	  WHEN day_of_week = 1 THEN 'Sunday'
	END
    AS Day_Week
FROM NULL_cleaned
),


-------------------------------------- Retain only ride ids with 16 alphanumerics with ride lengths above 1 minute --------------------------------------

clean_ride_id_data AS
(
	SELECT *
	FROM aggre_data
	WHERE LEN(ride_id) = 16 AND TotalMinutes > 1
),


-------------------------------------- Clean end & start stations names --------------------------------------

Clean_Table AS
(
	SELECT *,
	TRIM(REPLACE
		(REPLACE
			(end_station_name, '(*)',''),
				'(TEMP)','')) as cln_end_station_name,

	TRIM(REPLACE
		(REPLACE
			(start_station_name, '(*)',''),
				'(TEMP)','')) as cln_start_station_name

	FROM clean_ride_id_data
	WHERE end_station_name NOT LIKE '%(LBS-WH-TEST)%' and start_station_name NOT LIKE '%(LBS-WH-TEST)%' 
)

Select *
Into Final_Table
From Clean_Table

(3908105 rows affected)

Total execution time: 00:00:54.552

In [4]:
--Count the total number of Casual & Members

Select member_casual as User_type, count(member_casual) as Count
FROM Final_Table
group by member_casual

(2 rows affected)

Total execution time: 00:00:03.574

User_type,Count
member,2293609
casual,1614496


In [5]:
-- Daily trend of members

Select cast(started_at as date) as Rides_Date, member_casual as User_type, sum(TotalMinutes)/60 as total_ride
From Final_Table
group by cast(started_at as date), member_casual

(730 rows affected)

Total execution time: 00:00:06.704

Rides_Date,User_type,total_ride
2021-09-01,casual,3158
2021-09-02,casual,3863
2021-09-03,casual,4363
2021-09-04,casual,7822
2021-09-05,casual,12734
2021-09-06,casual,9354
2021-09-07,casual,2264
2021-09-08,casual,3180
2021-09-09,casual,3210
2021-09-10,casual,4469


In [6]:
-- Daily number of rides by week day 

Select cast(started_at as date) as Rides_Date, member_casual as User_type, count(ride_id) as num_rides
From Final_Table
group by cast(started_at as date), member_casual

(730 rows affected)

Total execution time: 00:00:07.015

Rides_Date,User_type,num_rides
2021-09-01,casual,8146
2021-09-02,casual,8651
2021-09-03,casual,9375
2021-09-04,casual,14341
2021-09-05,casual,18518
2021-09-06,casual,15215
2021-09-07,casual,5969
2021-09-08,casual,7632
2021-09-09,casual,8187
2021-09-10,casual,10125


In [7]:
-- Most visited stations
with station_names as
(
select cln_start_station_name + cln_end_station_name as station_name, member_casual
from Final_Table
)

select top 20 station_name, count(station_name) as Count_visits, member_casual as User_type
from station_names
group by station_name, member_casual
order by Count_visits desc

(20 rows affected)

Total execution time: 00:00:04.294

station_name,Count_visits,User_type
Streeter Dr & Grand AveStreeter Dr & Grand Ave,9704,casual
DuSable Lake Shore Dr & Monroe StDuSable Lake Shore Dr & Monroe St,6020,casual
DuSable Lake Shore Dr & Monroe StStreeter Dr & Grand Ave,5365,casual
Michigan Ave & Oak StMichigan Ave & Oak St,4402,casual
Ellis Ave & 60th StUniversity Ave & 57th St,4212,member
Ellis Ave & 60th StEllis Ave & 55th St,4076,member
University Ave & 57th StEllis Ave & 60th St,4069,member
Millennium ParkMillennium Park,3896,casual
Ellis Ave & 55th StEllis Ave & 60th St,3484,member
Streeter Dr & Grand AveDuSable Lake Shore Dr & Monroe St,2980,casual


In [8]:
-- Choice of ride by memebrship type
Select rideable_type, member_casual, COUNT(rideable_type) as count_rideable_type
From Final_table
group by rideable_type, member_casual

(5 rows affected)

Total execution time: 00:00:02.679

rideable_type,member_casual,count_rideable_type
classic_bike,member,1607083
docked_bike,casual,181621
electric_bike,member,686526
classic_bike,casual,892523
electric_bike,casual,540352
